In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # plotting
import biobss #EDA analysis

import os
import pickle

from scipy import signal
from scipy.signal import butter, iirnotch, lfilter
import scipy.signal
import scipy.ndimage
import scipy.stats as stats

from os import listdir
from os.path import isfile, join, isdir

from tqdm import tqdm

In [7]:
def get_edaindex(eda_base, sampling_rate):
    # Filtered signal
    t=np.arange(0,eda_base.size*(1/sampling_rate),(1/sampling_rate))
    t1=t[:eda_base.size]

    # Plot the original sampled data
    # plt.figure(figsize=(12,4))
    # plt.plot(t1,eda)
    # plt.xlabel('$Time (s)$') 
    # plt.ylabel('$EDA$') 
    # print(np.max(t1))

    #Filtered signal
    eda_f = biobss.edatools.filter_eda(eda_base, sampling_rate)

    # Plot the filtered data
    #plt.figure(figsize=(12,4))
    #plt.plot(t,eda_f)
    #plt.xlabel('$Time (s)$') 
    # plt.ylabel('$EDA$')
    
    # Decompose the signal in a tonic and phasic part
    eda = biobss.edatools.eda_decompose(eda_f, sampling_rate=sampling_rate)
    tonic = eda.iloc[:,0]
    phasic = eda.iloc[:,1]
    #print(eda)

    # Zero pad the signal to avoid errors
    zero_pad = 55*sampling_rate-1
    phasic = np.pad(phasic, (zero_pad, 0))
    phasic = np.insert(phasic,0,0)
    phasic = np.pad(phasic, (5, 0))
    t2 = np.arange(0,phasic.size*(1/sampling_rate),(1/sampling_rate))
    #print(phasic)
    #print(phasic.size)
    #print(t2.size)

    # Plot the zero padded signal
    #plt.figure(figsize=(12,4))
    #plt.plot(t2,phasic,label='phasic')
    ##plt.plot(t2,tonic,label='tonic')
    #plt.xlabel('$Time (s)$') 
    #plt.ylabel('$EDA$')

    # Peak detection
    peaks, properties = scipy.signal.find_peaks(phasic)
    heights, left_bases, right_bases = scipy.signal.peak_prominences(phasic, peaks)
    widths, width_heights, left_ips, __ = scipy.signal.peak_widths(phasic, peaks, rel_height=0.9)
    __, __, __, right_ips = scipy.signal.peak_widths(phasic, peaks, rel_height=0.5)

    heights1 = np.delete(heights, np.where(heights == np.max(heights)))
    heights2 = np.delete(heights1, np.where(heights1 == np.max(heights1)))

    # find the indices with an amplitude larger that 0.1
    keep = np.full(len(peaks), True)
    amplitude_min=0.2*np.max(heights2)
    time_min=50
    keep[heights<amplitude_min] = False

    # only keep those 
    peaks=peaks[keep]
    heights=heights[keep]
    widths=widths[keep]
    left_bases = left_bases[keep]
    right_bases = right_bases[keep]
    width_heights = width_heights[keep]
    left_ips = left_ips[keep]
    right_ips = right_ips[keep]

    #keep peaks inside a certain time interval
    keep1 = np.full(len(peaks), True)
    peak_diff = [0]
    for i in range(peaks.size-1):
        peak_diff_enum = t2[peaks[i+1]]-t2[peaks[i]]
        peak_diff.append(peak_diff_enum)

    keep1[np.array(peak_diff)>time_min] = False

    peaks=peaks[keep1]
    heights=heights[keep1]
    widths=widths[keep1]
    left_bases = left_bases[keep1]
    right_bases = right_bases[keep1]
    width_heights = width_heights[keep1]
    left_ips = left_ips[keep1]
    right_ips = right_ips[keep1]

    ## plot the peaks
    #plt.figure(figsize=(12,4))
    #plt.plot(t2,phasic,label='phasic')
    #plt.plot(t2[peaks],phasic[peaks],'o',label='peaks')
    ## labels and titles
    #plt.xlabel('$Time (s)$') 
    #plt.ylabel('$EDA$')
    #plt.legend()

    ##Taking the time interval between of the cluster of peaks with 5 sec extra
    tf = t2[peaks[0]-5*sampling_rate:peaks[-1]+5*sampling_rate] 
    phasic = phasic[peaks[0]-5*sampling_rate:peaks[-1]+5*sampling_rate]
    #print(tf.size)
    #print(phasic.size)

    # plot the interval
    #plt.figure(figsize=(12,4))
    #plt.plot(tf,phasic,label='phasic')
    ## labels and titles
    #plt.xlabel('$Time (s)$') 
    #plt.ylabel('$EDA$')
    #plt.legend()

    # Index samples interval
    start = peaks[1]-5*sampling_rate
    end = peaks[-1]+5*sampling_rate

    # Append all start and end indices to matrix
    # begin_l = np.append(begin_l, start[np.newaxis][np.newaxis], axis = 0)
    # end_l = np.append(end_l, end[np.newaxis][np.newaxis], axis = 0)
    #print(begin_l)
    #print(end_l)
    return tonic, phasic, start, end
    

In [1]:
#Features from EDA signal
def get_edafeatures(index, DF_feat, phasic, tonic, sampling_rate):
    tonic_phase_feat = biobss.edatools.from_decomposed(phasic, tonic, sampling_rate)
    #eda_feat = biobss.edatools.from_signal(eda_f, sampling_rate)
    #eda_freq_feat = biobss.edatools.eda_freq_features(eda)
    #eda_hjorth_feat = biobss.edatools.eda_hjorth_features(eda)
    #eda_sig_feat = biobss.edatools.eda_signal_features(eda)
    #eda_stat_feat = biobss.edatools.eda_stat_features(eda)

    #Features from tonic
    #tonic_feat = biobss.edatools.from_scl(tonic)

    ##Features from phasic
    #phasic_feat = biobss.edatools.from_scr(phasic)

    # Convert dictionary to Dataframe
    if DF_feat is None:
        DF_feat = pd.DataFrame(None, columns=tonic_phase_feat.keys())
        first = False

    # print(tonic_phase_feat.keys())
    DF_feat.loc[index] = tonic_phase_feat

    return DF_feat